In [1]:
from txtai.embeddings import Embeddings
from txtai.pipeline import Extractor, LLM
from txtai.pipeline import Textractor

import subprocess
import re
import os
import nltk

/opt/anaconda3/envs/envo/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/opt/anaconda3/envs/envo/lib/python3.8/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_list" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/anaconda3/envs/envo/lib/python3.8/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_name" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/opt/anaconda3/envs/envo/lib/python3.8/site-packages/pydantic/_internal/_fields.py:151: UserWarning: Field "model_group_alias" has conflict with protected namespace "model_".

Yo

In [2]:
llm = LLM("TheBloke/Mistral-7B-OpenOrca-GGUF/mistral-7b-openorca.Q4_K_M.gguf", verbose=False)

In [3]:
nltk.download('punkt')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/tahaelhihi/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [4]:
def stream(path):
  for f in sorted(os.listdir(path)):
    fpath = os.path.join(path, f)

    if f.endswith(("context994.txt")):
      print(f"Indexing {fpath}")
      yield textractor(fpath)

textractor = Textractor()


embeddings = Embeddings(content=True)
embeddings.index(stream("."))

2024-05-15 12:20:41,490 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar to /var/folders/l8/3t_1vg_s3szb10t1rf49zq2w0000gn/T/tika-server.jar.


Indexing ./clustcontext994.txt


2024-05-15 12:21:15,311 [MainThread  ] [INFO ]  Retrieving http://search.maven.org/remotecontent?filepath=org/apache/tika/tika-server-standard/2.6.0/tika-server-standard-2.6.0.jar.md5 to /var/folders/l8/3t_1vg_s3szb10t1rf49zq2w0000gn/T/tika-server.jar.md5.
2024-05-15 12:21:16,280 [MainThread  ] [WARNI]  Failed to see startup log message; retrying...


Indexing ./regcontext994.txt


In [5]:
def execute(question, context):
  prompt = f"""<|im_start|>system
{context}

<|im_end|>
<|im_start|>user
# Task:
Analyze the input sentence and extract necessary elements according to the instructions given above. 
Make sure the output strictly follows the specified format without any additional words or deviations.
Input:
{question}

<|im_end|>
<|im_start|>assistant
  """

  return llm(prompt, maxlength=4096)


In [6]:
def context(question):
  results = embeddings.search(question)
  if results:
      context = results[0]["text"]
  else:
      context = "No results found"
  return context

def rag(question):
  return execute(question, context(question))
     

In [7]:
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [8]:
def clean_intermediate(input_string):
    cleaned_string = input_string.replace('\n', ' ')
    cleaned_string = re.sub(r'\s+', ' ', cleaned_string)    
    cleaned_string = re.sub(r'[^\w\s]', '', cleaned_string.strip())
    cleaned_string = cleaned_string.lower()
    words = cleaned_string.split()
    return '[' + ', '.join(words) + ']'

In [9]:
def call_prolog_parser(input_string):
    command = [
        'swipl',  
        '-g',
        f"phrase(parse(Commands, Attributes), {input_string}), writeln(Commands), writeln(Attributes),halt",  
        '-t',
        'halt(0)',  
        'parser.pl'  
    ]

    result = subprocess.run(command, stdout=subprocess.PIPE, text=True)
    return result.stdout.strip()


In [10]:
def llm_to_AutoML_translator(query): 
    intermediate = rag(query)
    cleaned_intermediate = clean_intermediate(intermediate)
    output = call_prolog_parser(cleaned_intermediate)

    output = output.split('\n')

    print("intermediate:")
    print(intermediate)
    print('\n')
    
    print("AutoML Symbols:")
    print(output[0])
    
    print("Data columns:")
    print(output[1])

In [11]:
llm_to_AutoML_translator("Predict height based on volume through regression.")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


intermediate:
Regression:
   dependent attribute: height,
   independent attributes: volume.


AutoML Symbols:
[loadData,calculateVolume,splitTrainTestData,knnRegression,regressionMSEError]
Data columns:
[[],[surfaceInSquareMeters,heightInMeters],[],[[volume],height],knnRegression_OUTPUT]


In [12]:
llm_to_AutoML_translator("Given the objects dataset, how many types of objects can be found, according to their volume?")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


intermediate:
Clustering attributes: volume


AutoML Symbols:
[loadData,calculateVolume,clusterData,clusterCount]
Data columns:
[[],[surfaceInSquareMeters,heightInMeters],[volume],clusterData_OUTPUT]


In [13]:
llm_to_AutoML_translator("Train a model that can estimate volume based on surface given the objects dataset.") 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


intermediate:
Regression, dependent attribute: volume, independent attributes: surface.


AutoML Symbols:
[loadData,calculateVolume,splitTrainTestData,knnRegression,regressionMSEError]
Data columns:
[[],[surfaceInSquareMeters,heightInMeters],[],[[surface],volume],knnRegression_OUTPUT]


In [14]:
llm_to_AutoML_translator("How many different types of objects based on volume and height.") 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


intermediate:
Clustering attributes: volume, height.


AutoML Symbols:
[loadData,calculateVolume,clusterData,clusterCount]
Data columns:
[[],[surfaceInSquareMeters,heightInMeters],[volume,height],clusterData_OUTPUT]


In [15]:
llm_to_AutoML_translator("Train a model that can estimate mass based on surface given the objects dataset.") 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


intermediate:
Regression:
   dependent attribute: mass,
   independent attributes: surface


AutoML Symbols:
[loadData,splitTrainTestData,knnRegression,regressionMSEError]
Data columns:
[[],[],[[surface],mass],knnRegression_OUTPUT]


In [16]:
llm_to_AutoML_translator("Cluster objects based on mass.") 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


intermediate:
Clustering attributes: mass.


AutoML Symbols:
[loadData,clusterData,clusterCount]
Data columns:
[[],[mass],clusterData_OUTPUT]


In [17]:
llm_to_AutoML_translator("Train a model that can estimate volume based on surface and height given the objects dataset.") 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


intermediate:
Regression, dependent attribute: volume, independent attributes: mass, surface, height.


AutoML Symbols:
[loadData,calculateVolume,splitTrainTestData,knnRegression,regressionMSEError]
Data columns:
[[],[surfaceInSquareMeters,heightInMeters],[],[[mass,surface,height],volume],knnRegression_OUTPUT]


In [18]:
llm_to_AutoML_translator("Train a model that can estimate volume based on density given the objects dataset.") 

intermediate:
Regression, dependent attribute: volume, independent attributes: density.


AutoML Symbols:
[loadData,calculateVolume,calculateDensity,splitTrainTestData,knnRegression,regressionMSEError]
Data columns:
[[],[surfaceInSquareMeters,heightInMeters],[massInKilograms,surfaceInSquareMeters],[],[[density],volume],knnRegression_OUTPUT]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


In [20]:
llm_to_AutoML_translator("Predict mass using attributes such as surface and density given the objects dataset.") 

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


intermediate:
Regression, dependent attribute: mass, independent attributes: surface, density.


AutoML Symbols:
[loadData,calculateDensity,splitTrainTestData,knnRegression,regressionMSEError]
Data columns:
[[],[massInKilograms,surfaceInSquareMeters],[],[[surface,density],mass],knnRegression_OUTPUT]


In [21]:
llm_to_AutoML_translator("Train a model that can estimate density based on volume and mass given the objects dataset")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


intermediate:
Regression, dependent attribute: density, independent attributes: volume, mass.


AutoML Symbols:
[loadData,calculateDensity,calculateVolume,splitTrainTestData,knnRegression,regressionMSEError]
Data columns:
[[],[massInKilograms,surfaceInSquareMeters],[surfaceInSquareMeters,heightInMeters],[],[[volume,mass],density],knnRegression_OUTPUT]


In [23]:
llm_to_AutoML_translator("Train a model that can estimate density based on mass given the objects dataset")

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


intermediate:
Regression, dependent attribute: density, independent attributes: mass


AutoML Symbols:
[loadData,calculateDensity,splitTrainTestData,knnRegression,regressionMSEError]
Data columns:
[[],[massInKilograms,surfaceInSquareMeters],[],[[mass],density],knnRegression_OUTPUT]
